In [1]:
## Import the packages


import pandas as pd
from selenium import webdriver
import pickle

import time
from time import sleep
from numpy.random import uniform

from selenium.webdriver.common.keys import Keys

from difflib import SequenceMatcher


pd.options.display.max_rows = 50
pd.set_option('max_colwidth',400)
from selenium.common.exceptions import TimeoutException

import ast
import os

In [2]:
## Function to create random breaks


def sleep_r(length):
    if length == "s":
        return sleep(uniform(0.25, 0.5))
    elif length == "m":
        return sleep(uniform(1, 2))
    elif length == "l":
        return sleep(uniform(4, 6))
    else:
        print("error")


In [3]:
## Use your settings for Chromedriver, add executable path if you need it


chrome_options = webdriver.ChromeOptions()
executable_path='/Users/kristof/chromedriver'

chrome_options.add_argument('--load-extension=' + '/Users/kristof/Desktop/1.14.18_0/')

driver = webdriver.Chrome(chrome_options=chrome_options, executable_path=executable_path)






/miniconda3/envs/myPy/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: use options instead of chrome_options
  if __name__ == '__main__':


### ACM Digital Library is not easy to scrape due to various technical limitations of the page.
### This notebook explains how you can collect the links referring to the institutional profiles of author affiliations

### We use the a special format of the page that you can access by these steps:

1. Visit the site of special interest groups: https://dl.acm.org/sigs.cfm
2. Click on one 
3. Activate your browser's developer tool and select the network tool (Chrome: right click on the page, inspect and network)
4. Click on affiliations
5. Click on the element of the list that begins with "griddata"
6. A new browser tab will activate with the information that we need

### Copy the link below

https://dl.acm.org/griddata.cfc?method=getAffils&pageSize=20&useid=SP1530&myfilter=&_cf_ajaxproxytoken=5E9B9BF613AF9F1E959F91155D84EB&_cf_clientid=5379E73F83839820217137DFB0D9AEE1&_cf_rc=2&_cf_nodebug=true&_cf_nocache=true&returnFormat=json&_dc=1553621305258&start=0&limit=20&page=1&gridsortcolumn=&gridsortdir=ASC

### We will prepare a scraper that collects the data from the page, iterating on all many pages 

The end of the link contains the page number:

In [48]:
page=1&gridsortcolumn=&gridsortdir=ASC

'https://dl.acm.org/griddata.cfc?method=getAffils&pageSize=20&useid=SP1530&myfilter=&_cf_ajaxproxytoken=54B5A8C68E9F92FE95B08D8F8615B4&_cf_clientid=1AA6A394F54B569A74907B6588E1373A&_cf_rc=2&_cf_nodebug=true&_cf_nocache=true&returnFormat=json&_dc=1543841171853&start=0&limit=20&page=1&gridsortcolumn=&gridsortdir=ASC'

We will iterate by modifying the link (special link)

Check the number of pages in the original window (e.g. 30)

In [4]:
listdf=[]
page=1
special_link='https://dl.acm.org/griddata.cfc?method=getAffils&pageSize=20&useid=SP1530&myfilter=&_cf_ajaxproxytoken=54B5A8C68E9F92FE95B08D8F8615B4&_cf_clientid=1AA6A394F54B569A74907B6588E1373A&_cf_rc=2&_cf_nodebug=true&_cf_nocache=true&returnFormat=json&_dc=1543841171853&start=0&limit=20&'

#we will iterate 30 times 

while page<29:

    full_link=str(special_link)+'page='+str(page)+'&gridsortcolumn=&gridsortdir=ASC'
    driver.get(full_link)
    text=driver.find_element_by_tag_name('body').text
    dict_text=ast.literal_eval(text)

    cols=dict_text['QUERY']['COLUMNS']
    vals=dict_text['QUERY']['DATA']

    institution_list=[]
    link_list=[]
    pub_number_list=[]
    for i in vals:
        institution=i[0]
        institution_list.append(institution)
        link=i[2]
        link_list.append(link)
        pub=i[3]
        pub_number_list.append(pub)
    keys=['institutions', 'links', 'publication_n']
    values=[institution_list,link_list,pub_number_list]
    list_dict=dict(zip(keys, values))  
    df_dict=pd.DataFrame.from_dict(list_dict)
    listdf.append(df_dict)
    page=page+1
    sleep_r('l')
    



In [37]:
#listdf

In [28]:
df=pd.concat(listdf)

Df presents the institutions with the collected links

In [29]:
df

,institutions,links,publication_n
0,"University of Washington, Seattle","inst_page.cfm?id=SP1530&inst=60015481&filter=University of Washington, Seattle",345
1,Carnegie Mellon University,inst_page.cfm?id=SP1530&inst=60027950&filter=Carnegie Mellon University,270
2,Georgia Institute of Technology,inst_page.cfm?id=SP1530&inst=60019647&filter=Georgia Institute of Technology,193
3,"University of Maryland, Baltimore County","inst_page.cfm?id=SP1530&inst=60024997&filter=University of Maryland, Baltimore County",171
4,University of Dundee,inst_page.cfm?id=SP1530&inst=60008877&filter=University of Dundee,156
5,Rochester Institute of Technology,inst_page.cfm?id=SP1530&inst=60001777&filter=Rochester Institute of Technology,139
6,University of Maryland,inst_page.cfm?id=SP1530&inst=60020304&filter=University of Maryland,128
7,University of Manchester,inst_page.cfm?id=SP1530&inst=60003771&filter=University of Manchester,126
8,Stony Brook University,inst_page.cfm?id=SP1530&inst=60026415&filter=Stony Brook University,105
9,IBM Thomas J. Watson Research Center,inst_page.cfm?id=SP1530&inst=60017366&filter=IBM Thomas J. Watson Research Center,104


In [30]:
##Save the links to a csv file 
df.to_csv('acm_links_outside.csv')

### We have collected the links to the institutional profile. Let's collect the institutional profiles!

### This will have the following steps


1. Transform the collected links so that they can be opened in the browser
2. Open the link
3. Click on the paper section
4. Download the csv file with all paper metadata
5. Change the name of the file for the name of the institution

In [95]:
# save the profile csvs and change name for the institution

#Select your directory where the file will be downloaded
acm_dir = '/Users/kristof/Downloads'
os.chdir(acm_dir)
listdf=[]
for i in df['links']:
    temp=df.loc[df['links']==i].reset_index()
    real_name=temp['institutions']
    
    text=temp['links'][0]
   
    text='https://dl.acm.org/'+text


    text=text.replace('id=SP1530&', '').replace('inst=', '?id=')
    text=text.split('inst_page.cfm?')
    text=text[1].split('&?id=')
    text=text[0].split('&filter=')
    text='https://dl.acm.org/inst_page.cfm'+str(text[0])
    
    print(text)
    driver.get(text)
    while True:
        try:
            button=driver.find_element_by_id('tab-1019-btnEl').click()
            break
        except TimeoutException:
            print('not loaded 1')
            driver.refresh()
            sleep_r('l')
            
    name=driver.find_element_by_class_name('text14').find_element_by_tag_name('a').text
    name=name.replace('/', '-')
    
    sleep_r('m')
    sleep_r('m')
    
    
    while True:
        try:
            
            lists=driver.find_element_by_id('resultstats').find_element_by_id('resfound').find_element_by_id('searchtools').find_elements_by_tag_name('a')
            lists[3].click()
            break
        except:
            print('not loaded 2')
            driver.refresh()
            button=driver.find_element_by_id('tab-1019-btnEl').click()
            sleep_r('l')
        
    
    
    sleep_r('l')
    
    files = []
    while not files:
        try:
            files = filter(os.path.isfile, os.listdir(acm_dir))
            files = [os.path.join(acm_dir, f) for f in files]
            files.sort(key=lambda x: os.path.getmtime(x))
        except:
            sleep_r('m')
        
    
    newest_file = files[-1]
    os.rename(newest_file, str(name)+".csv")
    sleep_r('l')
    sleep_r('l')
    sleep_r('l')

https://dl.acm.org/inst_page.cfm?id=60015481
https://dl.acm.org/inst_page.cfm?id=60027950
not loaded 2
not loaded 2
not loaded 2
not loaded 2


KeyboardInterrupt: 